### 1. CREAR BASE DE DATOS

* Crear con código SQL una base de datos llamada supermercado.
* Código Python con mysql connector que ejecute el SQL para borrar y generar la base de datos vacía.

In [1]:
import pandas as pd
import mysql.connector as con
import random
import datetime
from datetime import timedelta

Base de datos Supermercado

In [2]:
# Crear base de datos "supermercado"
conexion = con.connect(
    host="localhost",
    port= 3306,
    user= "root",
    password= "admin"
)
cursor = conexion.cursor()
cursor.execute("DROP DATABASE IF EXISTS supermercado;")
cursor.execute("CREATE SCHEMA supermercado DEFAULT CHARACTER SET utf8;")
cursor.close()
conexion.close()

In [3]:
# Creo una sola tabla
# Conexion
conexion = con.connect(
    host="localhost",
    port= "3306",
    user= "root",
    password= "admin",
    database= "supermercado"
)
cursor = conexion.cursor()

# crear tabla
query_tabla_tiendas = """
CREATE TABLE IF NOT EXISTS tiendas (
    id_tienda INT AUTO_INCREMENT PRIMARY KEY,
    nombre_tienda VARCHAR(30),
    direccion_tienda VARCHAR(50),
    ciudad VARCHAR (50)
)
"""
# Hacer los cambios en la base de datos
cursor.execute(query_tabla_tiendas)
conexion.commit()
print("Tabla creada exitosamente.")

# Cerrar la conexion
cursor.close()
conexion.close()

Tabla creada exitosamente.


In [4]:
# Creo varias tablas a la vez
conexion = con.connect(
    host="localhost",
    port= "3306",
    user= "root",
    password= "admin",
    database= "supermercado"
)
cursor = conexion.cursor()

query_varias_tablas = ["""
    CREATE TABLE IF NOT EXISTS empleados (
    id_empleado INT AUTO_INCREMENT PRIMARY KEY,
    nombre_empleado VARCHAR(30),
    puesto VARCHAR(20),
    id_tienda INT,
    FOREIGN KEY (id_tienda) REFERENCES tiendas(id_tienda) ON DELETE CASCADE
)""",

    """
    CREATE TABLE IF NOT EXISTS categorias (
    id_categoria INT AUTO_INCREMENT PRIMARY KEY,
    nombre_categoria VARCHAR(40)
)""",

    """
    CREATE TABLE IF NOT EXISTS productos (
    id_producto INT AUTO_INCREMENT PRIMARY KEY,
    nombre_producto VARCHAR(40),
    precio DECIMAL (6, 2),
    stock INT,
    id_categoria INT,
    FOREIGN KEY (id_categoria) REFERENCES categorias(id_categoria) ON DELETE CASCADE
)""",

    """
    CREATE TABLE IF NOT EXISTS clientes (
    id_cliente INT AUTO_INCREMENT PRIMARY KEY,
    nombre VARCHAR(15),
    apellido VARCHAR (20),
    email VARCHAR (60),
    telefono INT,
    direccion VARCHAR (40),
    codigo_postal INT
)""",

    """
    CREATE TABLE IF NOT EXISTS ordenes (
    id_orden INT AUTO_INCREMENT PRIMARY KEY,
    id_cliente INT,
    FOREIGN KEY (id_cliente) REFERENCES clientes(id_cliente),
    id_empleado INT,
    FOREIGN KEY (id_empleado) REFERENCES empleados(id_empleado),
    fecha_orden DATE,
    metodo_pago ENUM('Tarjeta', 'Efectivo')
)""",

   """
    CREATE TABLE IF NOT EXISTS detalle_orden (
    id_detalle INT AUTO_INCREMENT PRIMARY KEY,
    id_orden INT NOT NULL,
    FOREIGN KEY (id_orden) REFERENCES ordenes(id_orden) ON DELETE CASCADE,
    id_producto INT NOT NULL,
    FOREIGN KEY (id_producto) REFERENCES productos(id_producto) ON DELETE CASCADE,
    cantidad INT (15) NOT NULL,
    descuento DECIMAL (3,2),
    precio_unitario DECIMAL(6,2) NOT NULL,
    precio_total DECIMAL(10, 2) NOT NULL
)"""
]
# Para ejecutar cada consulta a la base de datos y crear todas las tablas
for query in query_varias_tablas:
    cursor.execute(query)

conexion.commit()
print("Tablas creadas exitosamente.")

# Para copiar el "precio" de la tabla "productos" en la tabla "detalle_orden"
query_trigger = """
CREATE TRIGGER insertar_precio_unitario
BEFORE INSERT ON detalle_orden
FOR EACH ROW
BEGIN
    DECLARE precio_producto DECIMAL(6,2);

    -- Obtener el precio del producto desde la tabla productos
    SELECT precio INTO precio_producto 
    FROM productos 
    WHERE id_producto = NEW.id_producto;

    -- Asignar el precio al nuevo registro en detalle_orden
    SET NEW.precio_unitario = precio_producto;
END;
"""

cursor.execute(query_trigger)

conexion.commit()
print("Trigger creado exitosamente.")

# Para cerrar la conexión
cursor.close()
conexion.close()

Tablas creadas exitosamente.
Trigger creado exitosamente.


### 2. Generar datos demo desde Python

Tiendas

In [5]:
# Nombres de tiendas
filas = random.randint(11, 11)

nombre_tienda = [f'Tienda_0{numero}' if numero < 10 else f'Tienda_{numero}' for numero in range(filas)]
nombre_tienda

['Tienda_00',
 'Tienda_01',
 'Tienda_02',
 'Tienda_03',
 'Tienda_04',
 'Tienda_05',
 'Tienda_06',
 'Tienda_07',
 'Tienda_08',
 'Tienda_09',
 'Tienda_10']

In [6]:
# Direcciones 
direccion_tienda = [f'Calle_falsa_{numero}' if numero < 10 else f'Calle_falsa_{numero}' for numero in range(filas)]
direccion_tienda

['Calle_falsa_0',
 'Calle_falsa_1',
 'Calle_falsa_2',
 'Calle_falsa_3',
 'Calle_falsa_4',
 'Calle_falsa_5',
 'Calle_falsa_6',
 'Calle_falsa_7',
 'Calle_falsa_8',
 'Calle_falsa_9',
 'Calle_falsa_10']

In [7]:
# Lista de ciudades
ciudades = ["Madrid", "Barcelona", "Sevilla", "Valencia", "Bilbao", "Zaragoza", "Granada", "Alicante","Murcia","Albacete"]

# Número de ciudades aleatorias
num_ciudades = 10  

# Generar la lista de ciudades aleatorias
ciudades_aleatorias = [random.choice(ciudades) for _ in range(num_ciudades)]
ciudades_aleatorias

['Granada',
 'Alicante',
 'Bilbao',
 'Granada',
 'Sevilla',
 'Madrid',
 'Murcia',
 'Albacete',
 'Zaragoza',
 'Albacete']

In [8]:
# Insertamos los datos generados anteriormente
# Conectar a MySQL
conexion2 = con.connect(
    host="localhost",
    port= "3306",
    user="root",
    password="admin",
    database="supermercado"
)
cursor = conexion2.cursor()

# Insertamos los datos en la tabla
for i in range(10):
    cursor.execute("INSERT INTO tiendas (nombre_tienda, direccion_tienda, ciudad) VALUES (%s, %s, %s)",
                   (nombre_tienda[i], direccion_tienda[i], ciudades_aleatorias[i]))

conexion2.commit()
cursor.close()
conexion2.close()
print("Las tiendas han sido insertadas correctamente")

Las tiendas han sido insertadas correctamente


Empleados

In [9]:
# Lista de puestos
puestos = ["Director tienda", "Panadero", "Frutero", "Charcutero", "Pescatero", "Cajero", "Reponedor", "Repartidor"]

# Número de puestos aleatorias
num_puestos = 200  

# Generar la lista de puestos aleatorios
puestos_aleatorios = [random.choice(puestos) for _ in range(num_puestos)]
print(puestos_aleatorios)

['Cajero', 'Reponedor', 'Cajero', 'Director tienda', 'Pescatero', 'Frutero', 'Pescatero', 'Repartidor', 'Panadero', 'Director tienda', 'Reponedor', 'Reponedor', 'Director tienda', 'Panadero', 'Panadero', 'Director tienda', 'Repartidor', 'Repartidor', 'Reponedor', 'Charcutero', 'Repartidor', 'Charcutero', 'Pescatero', 'Charcutero', 'Frutero', 'Cajero', 'Director tienda', 'Pescatero', 'Panadero', 'Pescatero', 'Frutero', 'Reponedor', 'Charcutero', 'Charcutero', 'Pescatero', 'Pescatero', 'Repartidor', 'Cajero', 'Frutero', 'Cajero', 'Repartidor', 'Reponedor', 'Charcutero', 'Reponedor', 'Charcutero', 'Pescatero', 'Reponedor', 'Charcutero', 'Repartidor', 'Pescatero', 'Repartidor', 'Panadero', 'Pescatero', 'Cajero', 'Repartidor', 'Pescatero', 'Repartidor', 'Cajero', 'Charcutero', 'Pescatero', 'Panadero', 'Frutero', 'Reponedor', 'Frutero', 'Cajero', 'Pescatero', 'Reponedor', 'Director tienda', 'Frutero', 'Repartidor', 'Panadero', 'Director tienda', 'Charcutero', 'Repartidor', 'Cajero', 'Frutero

In [10]:
# Llenamos la tabla empleados
conexion4 = con.connect(
    host="localhost",
    port= "3306",
    user="root",
    password="admin",
    database="supermercado"
)
cursor = conexion4.cursor()

#Nombre empleados
nom_empleados = [f"Empleado{i}" for i in range(1, 201)]

# Lista de IDs de tiendas disponibles en la tabla "tiendas"
cursor.execute("SELECT id_tienda FROM tiendas")
id_tienda_disponibles = [row[0] for row in cursor.fetchall()]

# Inserto empleado con un puesto y un id_tienda aleatorio
for i in range(200):
    puesto_aleatorio = puestos_aleatorios[i]
    id_tienda_aleatorio = random.choice(id_tienda_disponibles)

    cursor.execute("INSERT INTO empleados (nombre_empleado, puesto, id_tienda) VALUES (%s, %s, %s)",
                   (nom_empleados[i],puesto_aleatorio, id_tienda_aleatorio))

conexion4.commit()
cursor.close()
conexion4.close()

print("Los 200 empleados han sido insertados correctamente con un puesto y una tienda aleatoria.")

Los 200 empleados han sido insertados correctamente con un puesto y una tienda aleatoria.


Categorias

In [11]:
# Insertar datos en la tabla categorias
import mysql.connector as con

# Conectar a MySQL
conexion5 = con.connect(
    host="localhost",
    port="3306",
    user="root",
    password="admin",
    database="supermercado"
)
cursor = conexion5.cursor()

# Lista de categorías
categorias = ["Carnes", "Lacteos", "Frutas", "Panaderia", "Bebidas", 
              "Cereales", "Congelados", "Conservas", "Limpieza", "Belleza"]

# Insertar las categorías en la tabla
for categoria in categorias:
    cursor.execute("INSERT INTO categorias (nombre_categoria) VALUES (%s)", (categoria,))

# Confirmar cambios y cerrar conexión
conexion5.commit()
cursor.close()
conexion5.close()

print("Las 10 categorias se han insertado correctamente")

Las 10 categorias se han insertado correctamente


Productos

In [12]:
# Conectar a MySQL
conexion6 = con.connect(
    host="localhost",
    port="3306",
    user="root",
    password="admin",
    database="supermercado"
)
cursor = conexion6.cursor()

# Diccionario de categorías y sus productos
productos_por_categoria = {
    "Lacteos": ["Leche", "Yogur griego natural", "Queso mozzarella", "Mantequilla"],
    "Carnes": ["Pechuga de pollo", "Carne picada", "Jamón", "Salchichas"],
    "Frutas": ["Manzana roja", "Plátano", "Zanahoria", "Lechuga"],
    "Panaderia": ["Pan integral", "Croissants", "Galletas", "Pastel de chocolate"],
    "Bebidas": ["Coca-Cola", "Zumo de naranja", "Café", "Agua mineral"],
    "Cereales": ["Arroz", "Lentejas", "Avena", "Fideos"],
    "Congelados": ["Helado de fresa", "Nuggets de pollo", "Espinacas congeladas", "Pizza congelada"],
    "Conservas": ["Atún", "Salsa de tomate", "Caballa", "Anchoas"],
    "Limpieza": ["Detergente líquido", "Jabón de manos", "Papel higiénico", "Desinfectante"],
    "Belleza": ["Champu", "Pasta de dientes", "Desodorante", "Crema hidratante"]
}

# IDs de las categorías existentes
cursor.execute("SELECT id_categoria, nombre_categoria FROM categorias")
categorias_disponibles = {nombre: id for id, nombre in cursor.fetchall()}

# Inserto productos con su categoría correspondiente
for categoria, lista_productos in productos_por_categoria.items():
    if categoria in categorias_disponibles:
        id_categoria = categorias_disponibles[categoria]
        for producto in lista_productos:
            precio = round(random.uniform(0.50, 50.00), 2)  # Precio aleatorio entre 1 y 50
            stock = random.randint(2, 100)  # Stock aleatorio entre 2 y 100 unidades
            
            cursor.execute("INSERT INTO productos (nombre_producto, precio, stock, id_categoria) VALUES (%s, %s, %s, %s)",
                           (producto, precio, stock, id_categoria))
    else:
        print(f"La categoría '{categoria}' no existe en la base de datos.")

conexion6.commit()
cursor.close()
conexion6.close()

print("Todos los productos han sido insertados correctamente con sus categorías correspondientes.")

Todos los productos han sido insertados correctamente con sus categorías correspondientes.


Clientes

In [13]:
# Llenamos la tabla clientes
conexion7 = con.connect(
    host="localhost",
    port= "3306",
    user="root",
    password="admin",
    database="supermercado"
)
cursor = conexion7.cursor()

#Nombres y apellidos clientes
nom_clientes = [f"Cliente{i}" for i in range(1, 2001)]
ape_clientes = [f"Apellido" for _ in range(2000)]

# email clientes
mail_clientes = [f"{nombre}.{apellido}@python.com" for nombre, apellido in zip(nom_clientes, ape_clientes)]

# Telefono clientes
tel_clientes = [random.randint(600000000, 699999999) for _ in range(2000)]

#Direccion clientes
dir_clientes = [f"Calle_cliente_{i}" for i in range(1, 2001)]

#Codigo postal
cod_postal = [random.randint(10000, 99999) for _ in range(2000)]


# Actualizar cada empleado con un puesto y un id_tienda aleatorio
for i in range(2000):
    cursor.execute("INSERT INTO clientes (nombre, apellido, email, telefono, direccion, codigo_postal) VALUES (%s, %s, %s, %s, %s, %s)",
                   (nom_clientes[i], ape_clientes[i], mail_clientes[i], tel_clientes[i], dir_clientes[i], cod_postal[i]))

conexion7.commit()
cursor.close()
conexion7.close()

print("Los 2000 clientes han sido insertados correctamente con todos sus datos.")

Los 2000 clientes han sido insertados correctamente con todos sus datos.


Ordenes

In [14]:
conexion8 = con.connect(
    host="localhost",
    port="3306",
    user="root",
    password="admin",
    database="supermercado"
)
cursor = conexion8.cursor()

# Obtener lista de ID de clientes
cursor.execute("SELECT id_cliente FROM clientes")
id_clientes = [row[0] for row in cursor.fetchall()]

# Obtener lista de ID de empleados
cursor.execute("SELECT id_empleado FROM empleados")
id_empleados = [row[0] for row in cursor.fetchall()]

# Generar 10,000 fechas aleatorias en 2024
fechas = [datetime.datetime(year=2024, month=1, day=1) + datetime.timedelta(days=random.randint(0, 364)) for _ in range(10000)]

# Insertar los datos en la tabla ordenes
for _ in range(10000): 
    id_cliente = random.choice(id_clientes) 
    id_empleado = random.choice(id_empleados) 
    fecha_orden = random.choice(fechas)  
    metodo_pago = random.choice(["Tarjeta", "Efectivo"])

    cursor.execute(
        "INSERT INTO ordenes (id_cliente, id_empleado, fecha_orden, metodo_pago) VALUES (%s, %s, %s, %s)",
        (id_cliente, id_empleado, fecha_orden, metodo_pago)
    )

conexion8.commit()
cursor.close()
conexion8.close()

print("Las 10,000 órdenes han sido insertadas correctamente")

Las 10,000 órdenes han sido insertadas correctamente


Detalles orden

In [15]:
from decimal import Decimal

# Conectar a la base de datos
conexion9 = con.connect(
    host="localhost",
    port=3306, 
    user="root",
    password="admin",
    database="supermercado"
)
cursor = conexion9.cursor()

# Obtener lista de ID de órdenes existentes
cursor.execute("SELECT id_orden FROM ordenes")
id_ordenes = [row[0] for row in cursor.fetchall()]

# Obtener lista de productos junto con su precio
cursor.execute("SELECT id_producto, precio FROM productos")
productos_info = {row[0]: row[1] for row in cursor.fetchall()}

# Inserto los datos SIN aplicar el descuento al precio total
for _ in range(30000): 
    id_orden = random.choice(id_ordenes)  # Escoge un ID de orden válido
    id_producto = random.choice(list(productos_info.keys()))  # Escoge un ID de producto válido
    cantidad = random.randint(1, 50)  # Genera una cantidad aleatoria entre 1 y 50
    precio_unitario = productos_info[id_producto]  # Obtiene el precio del producto
    precio_total = cantidad * precio_unitario  # Calcula el precio total

    # Generar un descuento aleatorio entre 0% y 15% y convertirlo a Decimal
    descuento = Decimal(str(round(random.uniform(0.00, 0.15), 2)))

    # Inserto los datos en la tabla detalle_orden SIN aplicar el descuento
    cursor.execute(
        "INSERT INTO detalle_orden (id_orden, id_producto, cantidad, precio_unitario, descuento, precio_total) VALUES (%s, %s, %s, %s, %s, %s)",
        (id_orden, id_producto, cantidad, precio_unitario, descuento, precio_total)
    )

# Confirmar los cambios
conexion9.commit()
cursor.close()
conexion9.close()

print("Se han insertado 30,000 registros con el descuento aplicado.")

Se han insertado 30,000 registros con el descuento aplicado.


### 3. Consultas SQL

1. Listado de ordenes con detalles de cliente y empleado
    * Muestra el ID de la orden, la fecha, el nombre del cliente, el nombre del empleado y el metodo de pago

In [25]:
conexion10 = con.connect(
    host="localhost",
    port=3306, 
    user="root",
    password="admin",
    database="supermercado"
)
cursor = conexion10.cursor()

query = """
SELECT 
    orden.id_orden, 
    orden.fecha_orden, 
    clien.nombre AS nombre_cliente, 
    emple.nombre_empleado, 
    orden.metodo_pago
FROM ordenes orden
JOIN clientes clien ON orden.id_cliente = clien.id_cliente
JOIN empleados emple ON orden.id_empleado = emple.id_empleado;
"""
# Ejecutamos la query
cursor.execute(query)

# Obtener los resultados
resultados = cursor.fetchall()


# Convierto los resultados a un DataFrame
df_ordenes = pd.DataFrame(resultados, columns=["ID Orden", "Fecha", "Cliente", "Empleado", "Método de Pago"])

print("Listado de ordenes")
print(df_ordenes)

conexion10.commit()
cursor.close()
conexion10.close()

Listado de ordenes
      ID Orden       Fecha      Cliente     Empleado Método de Pago
0          127  2024-04-22  Cliente1845    Empleado1       Efectivo
1          287  2024-01-21    Cliente41    Empleado1       Efectivo
2          435  2024-07-31    Cliente21    Empleado1       Efectivo
3          526  2024-04-30  Cliente1556    Empleado1       Efectivo
4          747  2024-03-26   Cliente580    Empleado1        Tarjeta
...        ...         ...          ...          ...            ...
9995      8776  2024-04-23  Cliente1854  Empleado200        Tarjeta
9996      8898  2024-08-06   Cliente843  Empleado200       Efectivo
9997      9107  2024-11-15    Cliente48  Empleado200        Tarjeta
9998      9173  2024-11-12   Cliente543  Empleado200       Efectivo
9999      9625  2024-09-28   Cliente162  Empleado200        Tarjeta

[10000 rows x 5 columns]


2. Productos con stock bajo
    * Filtra aquellos productos cuyo stock sea menos  a 10
    * Muestra el nombre del producto, categoria y stock

In [23]:
conexion11 = con.connect(
    host="localhost",
    port=3306, 
    user="root",
    password="admin",
    database="supermercado"
)
cursor = conexion11.cursor()

query = """
SELECT 
    produc.nombre_producto, 
    produc.stock, 
    cat.nombre_categoria 
FROM productos produc
JOIN categorias cat ON produc.id_categoria = cat.id_categoria
WHERE produc.stock < 10
"""
# Ejecutamos la query
cursor.execute(query)

# Obtener los resultados
resultados = cursor.fetchall()

df_stock = pd.DataFrame(resultados, columns=["Nombre Producto", "Stock", "Nombre Categoria"])

print("Productos con stock bajo")
print(df_stock)

conexion11.commit()
cursor.close()
conexion11.close()

Productos con stock bajo
   Nombre Producto  Stock Nombre Categoria
0     Pan integral      8        Panaderia
1  Helado de fresa      4       Congelados


3. Ventas totales por categoria
    * Muestra el nombre de la categoria y la suma total de las ventas

In [24]:
conexion12 = con.connect(
    host="localhost",
    port=3306, 
    user="root",
    password="admin",
    database="supermercado"
)
cursor = conexion12.cursor()

query = """
SELECT  
    cat.nombre_categoria,
    produc.precio, 
    SUM(deorden.cantidad),  -- Cantidad total vendida
    SUM(deorden.cantidad * produc.precio)  -- Coste total de ventas
FROM detalle_orden deorden
JOIN productos produc ON deorden.id_producto = produc.id_producto
JOIN categorias cat ON produc.id_categoria = cat.id_categoria
GROUP BY cat.nombre_categoria, produc.nombre_producto, produc.precio;
"""
# Ejecutamos la query
cursor.execute(query)

# Obtener los resultados
resultados = cursor.fetchall()

df_ventas = pd.DataFrame(resultados, columns=["Seccion", "Precio Unitario", "Cantidad", "Total ventas"])

print("Ventas totales por seccion")
print(df_ventas)

conexion12.commit()
cursor.close()
conexion12.close()

Ventas totales por seccion
       Seccion Precio Unitario Cantidad Total ventas
0       Carnes           46.87    18848    883405.76
1       Carnes           15.12    18524    280082.88
2       Carnes            0.95    18091     17186.45
3       Carnes           36.33    20189    733466.37
4      Lacteos           48.50    17895    867907.50
5      Lacteos           36.50    20105    733832.50
6      Lacteos           44.74    18045    807333.30
7      Lacteos            9.38    18337    172001.06
8       Frutas           19.38    18879    365875.02
9       Frutas           18.57    18822    349524.54
10      Frutas           47.20    18031    851063.20
11      Frutas            8.52    18313    156026.76
12   Panaderia           17.58    20410    358807.80
13   Panaderia           28.53    19327    551399.31
14   Panaderia           11.21    19102    214133.42
15   Panaderia           38.39    18128    695933.92
16     Bebidas           42.38    19347    819925.86
17     Bebidas     

4. Clientes con mayores gastos acumulados
    * Muestra el nombre del cliente y el monto total que ha gastado
    * Ten en cuenta los descuentos
    * Ordena de mayor a menor el gasto total

In [26]:
conexion13 = con.connect(
    host="localhost",
    port=3306, 
    user="root",
    password="admin",
    database="supermercado"
)
cursor = conexion13.cursor()

query = """
SELECT  
    clien.nombre, 
    SUM(deorde.precio_total - deorde.descuento) 
FROM detalle_orden deorde
JOIN ordenes orde ON deorde.id_orden = orde.id_orden
JOIN clientes clien ON orde.id_cliente = clien.id_cliente
GROUP BY clien.id_cliente, clien.nombre
ORDER BY SUM(deorde.precio_total - deorde.descuento) DESC;
"""

cursor.execute(query)

resultados = cursor.fetchall()

df_gastos = pd.DataFrame(resultados, columns=["Cliente", "Gasto Total"])
print("Cientes con mayores gastos")
print(df_gastos)

conexion13.commit()
cursor.close()
conexion13.close()

Cientes con mayores gastos
          Cliente Gasto Total
0     Cliente1568    37112.65
1     Cliente1942    31694.21
2      Cliente425    31497.78
3     Cliente1822    29281.32
4      Cliente892    28684.63
...           ...         ...
1971  Cliente1306      107.38
1972   Cliente985       81.91
1973    Cliente87       79.99
1974   Cliente756       43.05
1975   Cliente444        4.93

[1976 rows x 2 columns]


5. Empleados y numero de ordenes gestionadas
    * Muestra el nombre del empleado, el puesto y la cantidad de ordenes que ha gestionado
    * Utiliza GOUP BY Y COUNT

In [27]:
conexion14 = con.connect(
    host="localhost",
    port=3306, 
    user="root",
    password="admin",
    database="supermercado"
)
cursor = conexion14.cursor()

query = """
SELECT  
    empl.nombre_empleado, 
    empl.puesto, 
    COUNT(orde.id_orden) 
FROM empleados empl
JOIN ordenes orde ON empl.id_empleado = orde.id_empleado
GROUP BY empl.id_empleado, empl.nombre_empleado, empl.puesto
ORDER BY COUNT(orde.id_orden) DESC;
"""

cursor.execute(query)

resultados = cursor.fetchall()

df_empleados = pd.DataFrame(resultados, columns=["nombre_empleado", "puesto", "total_ordenes"])

print("Empleados que mas ordenes han gestionado")
print(df_empleados)

conexion14.commit()
cursor.close()
conexion14.close()

Empleados que mas ordenes han gestionado
    nombre_empleado           puesto  total_ordenes
0        Empleado48       Charcutero             71
1        Empleado95       Repartidor             70
2       Empleado127        Pescatero             64
3       Empleado141  Director tienda             64
4       Empleado152        Reponedor             64
..              ...              ...            ...
195     Empleado199        Reponedor             37
196      Empleado39          Frutero             36
197      Empleado19        Reponedor             35
198     Empleado143           Cajero             35
199      Empleado33       Charcutero             33

[200 rows x 3 columns]


6. Ordenes filtradas por fecha y tienda
    * Muestra todas las ordenes que se realizan en un rango de fechas determinado en una tienda
    * Incluye datos de la tienda y del cliente

In [28]:
conexion = con.connect(
    host="localhost",
    port=3306, 
    user="root",
    password="admin",
    database="supermercado"
)
cursor = conexion.cursor()

query = """
SELECT  
    orde.id_orden, 
    orde.fecha_orden, 
    clien.nombre, 
    clien.apellido, 
    clien.email, 
    tien.nombre_tienda, 
    tien.direccion_tienda, 
    tien.ciudad 
FROM ordenes orde
JOIN clientes clien ON orde.id_cliente = clien.id_cliente
JOIN empleados empl ON orde.id_empleado = empl.id_empleado
JOIN tiendas tien ON empl.id_tienda = tien.id_tienda
WHERE orde.fecha_orden BETWEEN '2024-10-01' AND '2024-10-31'
AND tien.id_tienda = 8
ORDER BY orde.fecha_orden ASC;
"""

cursor.execute(query)

resultados = cursor.fetchall()

# Convertir los datos a DataFrame
df_ordenes = pd.DataFrame(resultados, columns=[
    "Ticket Nº", "Fecha", "Nombre Cliente", "Apellido Cliente", "Email Cliente",
    "Tienda", "Direccion Tienda", "Ciudad"
])

print("Ordenes por fecha y tienda")
print(df_ordenes)

conexion.commit()
cursor.close()
conexion.close()

Ordenes por fecha y tienda
     Ticket Nº       Fecha Nombre Cliente Apellido Cliente  \
0         7210  2024-10-01    Cliente1732         Apellido   
1          290  2024-10-01    Cliente1189         Apellido   
2         5257  2024-10-02    Cliente1996         Apellido   
3         3321  2024-10-03     Cliente392         Apellido   
4         1258  2024-10-03     Cliente849         Apellido   
..         ...         ...            ...              ...   
102       2635  2024-10-31      Cliente43         Apellido   
103        637  2024-10-31     Cliente574         Apellido   
104        248  2024-10-31     Cliente761         Apellido   
105       1708  2024-10-31     Cliente949         Apellido   
106       9821  2024-10-31     Cliente204         Apellido   

                       Email Cliente     Tienda Direccion Tienda    Ciudad  
0    Cliente1732.Apellido@python.com  Tienda_07    Calle_falsa_7  Albacete  
1    Cliente1189.Apellido@python.com  Tienda_07    Calle_falsa_7  Albacete